In [8]:
import h2o
from h2o.automl import H2OAutoML

# Initializing the H2O server
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 53 secs
H2O_cluster_timezone:,Europe/Helsinki
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_User_h1gb80
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.981 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [10]:
# Loading the dataset
df = h2o.import_file('../data/df_prepped.csv')

# Dropping 'Countries' and 'Farm' columns from the  dataset
df = df.drop(['Countries', 'Farm'])


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
# Filter the rows where 'Year' is 2016
df_test = df[df['Year'] == 2016]

# Sort the filtered data by 'Year'
df_test = df_test.sort('Year')

# Filter the rows where 'Year' is not 2016
df_train = df[df['Year'] != 2016]


In [12]:
# Identifying predictors and target
y = 'Y_maize_major'
x = df.columns
x.remove(y)

In [13]:
# For categorical columns, ensuring they are handled as factors
#df_train['Countries'] = df_train['Countries'].asfactor()
#df_train['Farm'] = df_train['Farm'].asfactor()
#df_test['Countries'] = df_test['Countries'].asfactor()
#df_test['Farm'] = df_test['Farm'].asfactor()

In [14]:
# Running AutoML for 20 base models (limited here for example purposes)
aml = H2OAutoML(max_models=25, seed=1, nfolds=5)
aml.train(x=x, y=y, training_frame=df_train)

AutoML progress: |█
14:27:26.912: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),7/25
# GBM base models (used / total),7/12
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/10
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [15]:
# Viewing the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))  # Printing all rows instead of default (10 rows)

model_id                                                     rmse        mse       mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231217_142726     0.232637  0.0541198  0.133014  0.0747985                 0.0541198
GBM_4_AutoML_1_20231217_142726                           0.239758  0.0574838  0.137468  0.0766694                 0.0574838
StackedEnsemble_BestOfFamily_1_AutoML_1_20231217_142726  0.239788  0.0574982  0.137312  0.0765892                 0.0574982
GBM_3_AutoML_1_20231217_142726                           0.241516  0.0583299  0.141562  0.0780299                 0.0583299
GBM_grid_1_AutoML_1_20231217_142726_model_5              0.243887  0.0594809  0.137727  0.0779267                 0.0594809
GBM_2_AutoML_1_20231217_142726                           0.244667  0.059862   0.14574   0.0797588                 0.059862
GBM_5_AutoML_1_20231217_142726                           0.246988  0.061003   0.148772  0.0808488                 0.061003
GBM_grid_1

In [16]:
aml.leader

key,value
Stacking strategy,cross_validation
Number of base models (used / total),7/25
# GBM base models (used / total),7/12
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/10
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [17]:
# Predicting using the leader model on the test set
preds = aml.leader.predict(df_test)

# Combine the actual and predicted values
df_test_actual = df_test['Y_maize_major'].cbind(preds)

# Calculate performance on the test set
perf_test = aml.leader.model_performance(df_test_actual)

# Print MSE, RMSE, and MAE for the test set
print("Test Set Metrics:")
print("MSE:", perf_test.mse())
print("RMSE:", perf_test.rmse())
print("MAE:", perf_test.mae())


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test Set Metrics:
MSE: 3.661367557776199
RMSE: 1.913470030540379
MAE: 1.107242835074466


In [18]:
second_model_id = lb[1, 'model_id']
second_best_model = h2o.get_model(second_model_id)

# For predictions
preds = second_best_model.predict(df_test)

# For evaluating performance
perf = second_best_model.model_performance(df_test)
mse = perf.mse()
rmse = perf.rmse()
mae = perf.mae()

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
MSE: 0.11158106987728468
RMSE: 0.33403752764814415
MAE: 0.21429918373361928


In [19]:
third_model_id = lb[2, 'model_id']
third_best_model = h2o.get_model(third_model_id)

# For predictions
preds = third_best_model.predict(df_test)

# For evaluating performance
perf = third_best_model.model_performance(df_test)
mse = perf.mse()
rmse = perf.rmse()
mae = perf.mae()

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
MSE: 0.11212193123878514
RMSE: 0.334846130691076
MAE: 0.21496118558765373


In [20]:
#saving the model
h2o.save_model(model=aml.leader, path="./my_h2o_model", force=True)


'E:\\AUTOML\\AutoML_Project2\\notebooks\\my_h2o_model\\StackedEnsemble_AllModels_1_AutoML_1_20231217_142726'